## PINOUT - ESP8266

![pinout esp8266](images/ESP8266-NodeMCU-kit-12-E-pinout-gpio-pin.webp "Pinout ESP8266")

Referência: https://randomnerdtutorials.com/esp8266-pinout-reference-gpios/

https://wokwi.com/projects/378849911430132737

![HC-SR04](images/ESP8266-Ultrasonic-Sensor-Wiring-Fritzing-Diagram.webp "HC-SR04")

Referência: https://randomnerdtutorials.com/micropython-hc-sr04-ultrasonic-esp32-esp8266/

In [21]:
%serialconnect to --port=/dev/ttyUSB0 --baud=115200

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [22]:
import machine, time
from machine import Pin

__version__ = '0.2.0'
__author__ = 'Roberto Sánchez'
__license__ = "Apache License 2.0. https://www.apache.org/licenses/LICENSE-2.0"

class HCSR04:
    """
    Driver to use the untrasonic sensor HC-SR04.
    The sensor range is between 2cm and 4m.
    The timeouts received listening to echo pin are converted to OSError('Out of range')
    """
    # echo_timeout_us is based in chip range limit (400cm)
    def __init__(self, trigger_pin, echo_pin, echo_timeout_us=500*2*30):
        """
        trigger_pin: Output pin to send pulses
        echo_pin: Readonly pin to measure the distance. The pin should be protected with 1k resistor
        echo_timeout_us: Timeout in microseconds to listen to echo pin. 
        By default is based in sensor limit range (4m)
        """
        self.echo_timeout_us = echo_timeout_us
        # Init trigger pin (out)
        self.trigger = Pin(trigger_pin, mode=Pin.OUT, pull=None)
        self.trigger.value(0)

        # Init echo pin (in)
        self.echo = Pin(echo_pin, mode=Pin.IN, pull=None)

    def _send_pulse_and_wait(self):
        """
        Send the pulse to trigger and listen on echo pin.
        We use the method `machine.time_pulse_us()` to get the microseconds until the echo is received.
        """
        self.trigger.value(0) # Stabilize the sensor
        time.sleep_us(5)
        self.trigger.value(1)
        # Send a 10us pulse.
        time.sleep_us(10)
        self.trigger.value(0)
        try:
            pulse_time = machine.time_pulse_us(self.echo, 1, self.echo_timeout_us)
            return pulse_time
        except OSError as ex:
            if ex.args[0] == 110: # 110 = ETIMEDOUT
                raise OSError('Out of range')
            raise ex

    def distance_mm(self):
        """
        Get the distance in milimeters without floating point operations.
        """
        pulse_time = self._send_pulse_and_wait()

        # To calculate the distance we get the pulse_time and divide it by 2 
        # (the pulse walk the distance twice) and by 29.1 becasue
        # the sound speed on air (343.2 m/s), that It's equivalent to
        # 0.34320 mm/us that is 1mm each 2.91us
        # pulse_time // 2 // 2.91 -> pulse_time // 5.82 -> pulse_time * 100 // 582 
        mm = pulse_time * 100 // 582
        return mm

    def distance_cm(self):
        """
        Get the distance in centimeters with floating point operations.
        It returns a float
        """
        pulse_time = self._send_pulse_and_wait()

        # To calculate the distance we get the pulse_time and divide it by 2 
        # (the pulse walk the distance twice) and by 29.1 becasue
        # the sound speed on air (343.2 m/s), that It's equivalent to
        # 0.034320 cm/us that is 1cm each 29.1us
        cms = (pulse_time / 2) / 29.1
        return cms

In [23]:
# Complete project details at https://RandomNerdTutorials.com/micropython-hc-sr04-ultrasonic-esp32-esp8266/
#from hcsr04 import HCSR04
from time import sleep

# ESP32
#sensor = HCSR04(trigger_pin=5, echo_pin=18, echo_timeout_us=10000)

# ESP8266
sensor = HCSR04(trigger_pin=12, echo_pin=14, echo_timeout_us=10000)

while True:
    distance = sensor.distance_cm()
    print('Distance:', distance, 'cm')
    sleep(1)

Distance: 50.7388 cm
Distance: 52.3024 cm
Distance: 3.98625 cm
Distance: 16.2887 cm
Distance: 54.0206 cm
Distance: 50.4983 cm
Distance: 27.8522 cm
.Distance: 51.3058 cm
Distance: 40.4124 cm
Distance: 40.7732 cm
Distance: 44.6048 cm
Distance: 37.1649 cm
.Distance: 100.017 cm
Distance: 143.849 cm
Distance: 144.261 cm
Distance: 144.244 cm
Distance: 104.124 cm
.Distance: 94.2096 cm
Distance: 82.7663 cm
Distance: 55.3608 cm
Distance: 63.1443 cm
Distance: 65.378 cm
.Distance: 52.268 cm
Distance: 16.0997 cm
Distance: 14.4674 cm
Distance: 10.2062 cm
Distance: 8.8488 cm
Distance: 36.6495 cm
Distance: 35.4983 cm
Distance: 20.4296 cm
.Distance: 22.8522 cm
Distance: 19.9313 cm
Distance: 19.244 cm
Distance: 16.6838 cm
Distance: 11.1856 cm
Distance: 16.4948 cm
Distance: 16.5808 cm
Distance: 15.2749 cm
Distance: 15.2406 cm
Distance: 18.0928 cm
.Distance: 32.8694 cm
Distance: 39.4845 cm
Distance: 27.5601 cm
Distance: 28.7113 cm
Distance: 43.2646 cm
.Distance: 41.6667 cm
Distance: 63.0069 cm
Distance: 